In [2]:
# importing libraries
from PyQt5.QtCore import *
from PyQt5.QtWidgets import *
from PyQt5.QtGui import *
import random
import sys
 
# creating game window
 
 
class Window(QMainWindow):
    def __init__(self):
        super(Window, self).__init__()
 
        # creating a board object
        self.board = Board(self)
 
        # creating a status bar to show result
        self.statusbar = self.statusBar()
 
        # adding border to the status bar
        self.statusbar.setStyleSheet("border: 2px solid black")
 
        # calling showMessage method when signal received by board
        self.board.msg2statusbar[str].connect(self.statusbar.showMessage)
 
        # adding board as a central widget
        self.setCentralWidget(self.board)
 
        # setting title to the window
        self.setWindowTitle('Snake game')
 
        # setting geometry to the window
        self.setGeometry(100, 100, 600, 400)
 
        # starting the board object
        self.board.start()
 
        # showing the main window
        self.show()
 
# creating a board class
# that inherits QFrame
 
 
class Board(QFrame):
 
    # creating signal object
    msg2statusbar = pyqtSignal(str)
 
    # speed of the snake
    # timer countdown time
    SPEED = 80
 
    # block width and height
    WIDTHINBLOCKS = 60
    HEIGHTINBLOCKS = 40
 
    # constructor
    def __init__(self, parent):
        super(Board, self).__init__(parent)
 
        # creating a timer
        self.timer = QBasicTimer()
 
        # snake
        self.snake = [[5, 10], [5, 11]]
 
        # current head x head
        self.current_x_head = self.snake[0][0]
        # current y head
        self.current_y_head = self.snake[0][1]
 
        # food list
        self.food = []
 
        # growing is false
        self.grow_snake = False
 
        # board list
        self.board = []
 
        # direction
        self.direction = 1
 
        # called drop food method
        self.drop_food()
 
        # setting focus
        self.setFocusPolicy(Qt.StrongFocus)
 
    # square width method
    def square_width(self):
        return self.contentsRect().width() / Board.WIDTHINBLOCKS
 
    # square height
    def square_height(self):
        return self.contentsRect().height() / Board.HEIGHTINBLOCKS
 
    # start method
    def start(self):
        # msg for status bar
        # score = current len - 2
        self.msg2statusbar.emit(str(len(self.snake) - 2))
 
        # starting timer
        self.timer.start(Board.SPEED, self)
 
    # paint event
    def paintEvent(self, event):
 
        # creating painter object
        painter = QPainter(self)
 
        # getting rectangle
        rect = self.contentsRect()
 
        # board top
        boardtop = rect.bottom() - Board.HEIGHTINBLOCKS * self.square_height()
 
        # drawing snake
        for pos in self.snake:
            self.draw_square(painter, rect.left() + pos[0] * self.square_width(),
                             boardtop + pos[1] * self.square_height())
 
        # drawing food
        for pos in self.food:
            self.draw_square(painter, rect.left() + pos[0] * self.square_width(),
                             boardtop + pos[1] * self.square_height())
 
    # drawing square
    def draw_square(self, painter, x, y):
        # color
        color = QColor(0x228B22)
 
        # painting rectangle
        painter.fillRect(x + 1, y + 1, self.square_width() - 2,
                         self.square_height() - 2, color)
 
    # key press event
    def keyPressEvent(self, event):
 
        # getting key pressed
        key = event.key()
 
        # if left key pressed
        if key == Qt.Key_Left:
            # if direction is not right
            if self.direction != 2:
                # set direction to left
                self.direction = 1
 
        # if right key is pressed
        elif key == Qt.Key_Right:
            # if direction is not left
            if self.direction != 1:
                # set direction to right
                self.direction = 2
 
        # if down key is pressed
        elif key == Qt.Key_Down:
            # if direction is not up
            if self.direction != 4:
                # set direction to down
                self.direction = 3
 
        # if up key is pressed
        elif key == Qt.Key_Up:
            # if direction is not down
            if self.direction != 3:
                # set direction to up
                self.direction = 4
 
    # method to move the snake
    def move_snake(self):
 
        # if direction is left change its position
        if self.direction == 1:
            self.current_x_head, self.current_y_head = self.current_x_head - 1, self.current_y_head
 
            # if it goes beyond left wall
            if self.current_x_head < 0:
                self.current_x_head = Board.WIDTHINBLOCKS - 1
 
        # if direction is right change its position
        if self.direction == 2:
            self.current_x_head, self.current_y_head = self.current_x_head + 1, self.current_y_head
            # if it goes beyond right wall
            if self.current_x_head == Board.WIDTHINBLOCKS:
                self.current_x_head = 0
 
        # if direction is down change its position
        if self.direction == 3:
            self.current_x_head, self.current_y_head = self.current_x_head, self.current_y_head + 1
            # if it goes beyond down wall
            if self.current_y_head == Board.HEIGHTINBLOCKS:
                self.current_y_head = 0
 
        # if direction is up change its position
        if self.direction == 4:
            self.current_x_head, self.current_y_head = self.current_x_head, self.current_y_head - 1
            # if it goes beyond up wall
            if self.current_y_head < 0:
                self.current_y_head = Board.HEIGHTINBLOCKS
 
        # changing head position
        head = [self.current_x_head, self.current_y_head]
        # inset head in snake list
        self.snake.insert(0, head)
 
        # if snake grow is False
        if not self.grow_snake:
            # pop the last element
            self.snake.pop()
 
        else:
            # show msg in status bar
            self.msg2statusbar.emit(str(len(self.snake)-2))
            # make grow_snake to false
            self.grow_snake = False
 
    # time event method
    def timerEvent(self, event):
 
        # checking timer id
        if event.timerId() == self.timer.timerId():
 
            # call move snake method
            self.move_snake()
            # call food collision method
            self.is_food_collision()
            # call is suicide method
            self.is_suicide()
            # update the window
            self.update()
 
    # method to check if snake collides itself
    def is_suicide(self):
        # traversing the snake
        for i in range(1, len(self.snake)):
            # if collision found
            if self.snake[i] == self.snake[0]:
                # show game ended msg in status bar
                self.msg2statusbar.emit(str("Game Ended"))
                # making background color black
                self.setStyleSheet("background-color: black")
                # stopping the timer
                self.timer.stop()
                # updating the window
                self.update()
 
    # method to check if the food cis collied
    def is_food_collision(self):
 
        # traversing the position of the food
        for pos in self.food:
            # if food position is similar of snake position
            if pos == self.snake[0]:
                # remove the food
                self.food.remove(pos)
                # call drop food method
                self.drop_food()
                # grow the snake
                self.grow_snake = True
 
    # method to drop food on screen
    def drop_food(self):
        # creating random co-ordinates
        x = random.randint(3, 58)
        y = random.randint(3, 38)
 
        # traversing if snake position is not equal to the
        # food position so that food do not drop on snake
        for pos in self.snake:
            # if position matches
            if pos == [x, y]:
                # call drop food method again
                self.drop_food()
 
        # append food location
        self.food.append([x, y])
 
 
# main method
if __name__ == '__main__':
    app = QApplication([])
    window = Window()
    sys.exit(app.exec_())

C:\Users\Artur.Abaidulov\AppData\Local\Temp\ipykernel_1128\4017630389.py:137: DeprecationWarning: an integer is required (got type float).  Implicit conversion to integers using __int__ is deprecated, and may be removed in a future version of Python.
  painter.fillRect(x + 1, y + 1, self.square_width() - 2,


SystemExit: 0

c:\Users\Artur.Abaidulov\AppData\Local\Programs\Python\Python39\lib\site-packages\IPython\core\interactiveshell.py:3441: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


In [3]:
# importing libraries
from PyQt5.QtWidgets import *
from PyQt5 import QtCore, QtGui
from PyQt5.QtGui import *
from PyQt5.QtCore import *
import random
import sys


class Window(QMainWindow):

	def __init__(self):
		super().__init__()

		# setting title
		self.setWindowTitle("Python ")

		# setting geometry
		self.setGeometry(100, 100, 500, 500)

		# calling method
		self.UiComponents()

		# showing all the widgets
		self.show()

		# counter
		self.count_value = 30

		# score
		self.score_value = 0

		# start flag
		self.start_Flag = False

		# list of possible colour.
		self.color_list = ['Red', 'Blue', 'Green', 'Pink', 'Black',
				'Yellow', 'Orange', 'Purple', 'Brown']


		# method for components
	def UiComponents(self):

		# creating head label
		head = QLabel("Color Game", self)

		# setting geometry to the head
		head.setGeometry(100, 10, 300, 60)

		# font
		font = QFont('Times', 14)
		font.setBold(True)
		font.setItalic(True)
		font.setUnderline(True)

		# setting font to the head
		head.setFont(font)

		# setting alignment of the head
		head.setAlignment(Qt.AlignCenter)

		# instruction label
		instruction = QLabel("Instruction : Enter the Color not the text. "
							"Press Start button to start the game		 "
							"Note : Time limit for game is 30 seconds", self)

		# making it multi line
		instruction.setWordWrap(True)

		# setting geometry to the label
		instruction.setGeometry(20, 60, 460, 60)

		# creating start button
		start = QPushButton("Start / Reset", self)

		# setting geometry to the push button
		start.setGeometry(200, 120, 100, 35)

		# adding action to the start button
		start.clicked.connect(self.start_action)

		# creating a score label
		self.score = QLabel("Score : 0", self)

		# setting geometry
		self.score.setGeometry(160, 170, 180, 50)

		# setting alignment
		self.score.setAlignment(Qt.AlignCenter)

		# setting font
		self.score.setFont(QFont('Times', 16))

		# setting style sheet
		self.score.setStyleSheet("QLabel"
								"{"
								"border : 2px solid black;"
								"color : green;"
								"background : lightgrey;"
								"}")

		# creating label to show color
		self.color = QLabel("Color Name", self)

		# setting geometry
		self.color.setGeometry(50, 230, 400, 120)

		# setting alignment
		self.color.setAlignment(Qt.AlignCenter)

		# setting font
		self.color.setFont(QFont('Times', 30))

		# creating a line edit
		self.input_text = QLineEdit(self)

		# setting geometry
		self.input_text.setGeometry(150, 340, 200, 50)

		# setting font
		self.input_text.setFont(QFont('Arial', 14))

		# making line edit disabled
		self.input_text.setDisabled(True)

		# adding action to it when enter is pressed
		self.input_text.returnPressed.connect(self.input_action)

		# creating a timer label
		self.count = QLabel("30", self)

		# setting geometry
		self.count.setGeometry(225, 430, 50, 50)

		# setting alignment
		self.count.setAlignment(Qt.AlignCenter)

		# setting font
		self.count.setFont(QFont('Times', 14))

		# setting style sheet
		self.count.setStyleSheet("border : 2px solid black;"
								"background : lightgrey;")

		# creating a timer object
		timer = QTimer(self)

		# adding action to the timer
		timer.timeout.connect(self.show_time)

		# start timer
		timer.start(1000)

	def show_time(self):

		if self.start_Flag:

			# showing count value to label
			self.count.setText(str(self.count_value))

			# checking if count value is zero
			if self.count_value == 0:

				# making start flag to false
				self.start_Flag = False

				# making line edit widget disable
				self.input_text.setDisabled(True)


			# decrementing the count value
			self.count_value -= 1




	def start_action(self):

		# making start flag true
		self.start_Flag = True

		# resetting score
		self.score.setText("Score : 0")
		self.score_value = 0

		# resetting count value
		self.count_value = 30

		# clearing line edit text
		self.input_text.clear()

		# making line edit enabled
		self.input_text.setEnabled(True)

		# getting random color
		self.random_color = random.choice(self.color_list)

		# making color choice random color
		self.random_color.lower()

		# setting random color to the label
		self.color.setStyleSheet("color : " + self.random_color + ";")

		# getting another random color name
		random_text = random.choice(self.color_list)

		# setting text to label
		self.color.setText(random_text)



	def input_action(self):

		# get the line edit text
		text = self.input_text.text()

		# making text lower case
		text.lower()

		# checking text with random color
		if text == self.random_color:
			# clearing line edit text
			self.input_text.clear()

			# incrementing score value
			self.score_value += 1

			# setting score to the score label
			self.score.setText("Score : " + str(self.score_value))

			# getting random color
			self.random_color = random.choice(self.color_list)

			# making color choice random color
			self.random_color.lower()

			# setting random color to the label
			self.color.setStyleSheet("color : " + self.random_color + ";")

			# getting another random color name
			random_text = random.choice(self.color_list)

			# setting text to label
			self.color.setText(random_text)



# create pyqt5 app
App = QApplication(sys.argv)

# create the instance of our Window
window = Window()

# start the app
sys.exit(App.exec())


SystemExit: 0

In [1]:
# importing the required libraries
from PyQt5.QtGui import * 
from PyQt5.QtWidgets import * 
import sys
  
class Window(QMainWindow):
    def __init__(self):
        super().__init__()
  
        # set the title
        self.setWindowTitle("Geometry")
  
        # setting  the geometry of window
        # setGeometry(left, top, width, height)
        self.setGeometry(100, 60, 1000, 800)
  
        # creating a label widget
        self.widget = QLabel('Hello', self)
  
  
  
        # show all the widgets
        self.show()
  
  
  
# create pyqt5 app
App = QApplication(sys.argv)
  
# create the instance of our Window
window = Window()
# start the app
sys.exit(App.exec())

SystemExit: 0

c:\Users\Artur.Abaidulov\AppData\Local\Programs\Python\Python39\lib\site-packages\IPython\core\interactiveshell.py:3441: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)
